In [1]:
%load_ext autoreload
%autoreload 2
import torch, rdkit
import sys, pathlib
from pathlib import Path
PROJECT_ROOT = Path.home()/"바탕화면"/"torch"/"Chem"
sys.path.insert(0, str(PROJECT_ROOT))
from rdkit import Chem
from utils.utils import *


from tqdm import trange
from pathlib import Path
device   = "cuda"

vocab = {'(': 0, ')': 1, '=': 2, '[O;!R;C]': 3, '[CH3;!R;C]': 4, '[CH2;!R;CN]': 5, '[CH2;!R;CC]': 6, '[CH2;!R;CO]': 7, '[*;!R;C]': 8, '[SOS]': 9, '[EOS]': 10, '[O;!R;CC]': 11, '[NH;!R;CC]': 12, '[*;!R;O]': 13, '[O;!R;*C]': 14, '[C;!R;*OO]': 15, '[CH;!R;CCO]': 16, '[CH;!R;CCN]': 17, '[C;!R;CNO]': 18, '[N;!R;CCC]': 19, '[C;!R;*NO]': 20, '[NH;!R;*C]': 21, '[*;!R;N]': 22, '[CH3;!R;N]': 23, '[CH;!R;CCC]': 24, '[C;!R;CCCO]': 25, '[CH;!R;CC]': 26, '[F;!R;C]': 27, '[CH2;!R;C]': 28, '[C;!R;COO]': 29, '[CH3;!R;O]': 30, '[C;!R;CCCN]': 31, '#': 32, '[C;!R;CCCC]': 33, '[CH2;!R;CS]': 34, '[C;!R;CC]': 35, '[OH;!R;C]': 36, '[S;!R;CC]': 37, '[N;!R;C]': 38, '[C;!R;CN]': 39, '[CH;!R;C]': 40, '[C;!R;CCC]': 41, '[NH2;!R;C]': 42, '[CH;!R;CFF]': 43, '[O;!R;S]': 44, '[O;!R;CN]': 45, '[CH;!R;CCS]': 46, '[CH2;!R;CF]': 47, '[CH3;!R;S]': 48, '[C;!R;CCO]': 49, '[NH;!R;CO]': 50, '[CH2;!R;*C]': 51, '[C;!R;NNO]': 52, '[C;!R;CFFF]': 53, '[C;!R;CCFF]': 54, '[C;!R;NOO]': 55, '[C;!R;OOO]': 56, '[S;!R;CCOO]': 57, '[CH3;!R;Si]': 58, '[C;!R;NNS]': 59, '[S;!R;C]': 60, '[NH;!R;C]': 61, '[Cl;!R;C]': 62, '[OH;!R;P]': 63, '1': 64, '[C;!R;CNN]': 65, '[CH;!R;CN]': 66, '[SH;!R;C]': 67, '[CH;!R;CCF]': 68, '[O;!R;P]': 69, '[CH2;!R;NO]': 70, '[O;R;CC]': 71, '[C;!R;CCCl]': 72, '[CH2;!R;CSi]': 73, '[CH;!R;NN]': 74, '[cH;R;CC]': 75, '[c;R;CCO]': 76, '[CH;!R;CO]': 77, '[Si;!R;CCCC]': 78, '[C;!R;CCN]': 79, '[N;!R;CN]': 80, '[NH;!R;CS]': 81, '[CH;!R;COO]': 82, '[N;!R;CCO]': 83, '[P;!R;COOO]': 84, '[C;!R;CCNN]': 85, '[N;!R;CO]': 86, '[CH;!R;CCl]': 87, '[S;!R;CNOO]': 88, '[C;!R;CCCF]': 89, '[CH2;!R;NSi]': 90, '[O;!R;CS]': 91, '[NH2;!R;O]': 92, '[NH;!R;CN]': 93, '[CH;R;CCO]': 94, '[CH2;R;CO]': 95, '[C;R;OOO]': 96, '[C;!R;CCF]': 97, '[C;!R;CCOO]': 98, '[N;!R;CCN]': 99, '[N;!R;CS]': 100, '[S;!R;CCNO]': 101, '[OH;!R;N]': 102, '[C;!R;NNN]': 103, '[S;!R;CCO]': 104, '[CH2;!R;CP]': 105, '[CH;!R;CNP]': 106, '[OH;!R;S]': 107, '[NH2;!R;N]': 108, '[N;!R;CC]': 109, '[Si;!R;*CCC]': 110, '[*;!R;Si]': 111, '[CH;!R;CS]': 112, '[C;!R;*CO]': 113, '[S;!R;COOO]': 114, '[O;!R;CP]': 115, '[S;!R;NNOO]': 116, '[NH2;!R;S]': 117, '[CH;R;*CN]': 118, '[N;R;CCC]': 119, '2': 120, '[CH2;!R;OO]': 121, '[C;!R;NN]': 122, 'N': 123, '[S;!R;COOS]': 124, '[S;!R;S]': 125, '[C;!R;CCCS]': 126, '[CH2;!R;NS]': 127, '[P;!R;OOOO]': 128, '[CH;!R;CNO]': 129, '[S;!R;OOO]': 130, '[O;!R;*S]': 131, '[CH2;!R;SS]': 132, '[CH;!R;FFO]': 133, '[CH2;!R;OP]': 134, '[C;!R;NSS]': 135, '[CH;!R;COP]': 136, '[C;!R;CFFO]': 137, '[CH;!R;CNS]': 138, '[c;R;CCC]': 139, '[CH;!R;CNN]': 140, '[C;!R;CSi]': 141, '[C;R;NNO]': 142, '[c;R;*CC]': 143, '[CH;!R;*CC]': 144, '[CH2;!R;NN]': 145, '[[N+]': 146, 'O': 147, '[[O-]': 148, '[N;!R;NO]': 149, '[CH;!R;NO]': 150, '[CH2;!R;OSi]': 151, '[CH;!R;NNN]': 152, '[P;!R;CCOO]': 153, '[C;!R;CClCl]': 154, '[C;!R;CCNO]': 155, '[C;!R;COS]': 156, '[N;!R;CCS]': 157, '[C;!R;CCCCl]': 158, '[C;!R;CCFO]': 159, '[C;!R;CCOP]': 160, '[SH;!R;O]': 161, '[S;!R;OOOO]': 162, '[C;!R;OSS]': 163, '[C;!R;CCClCl]': 164, '[PH;!R;CCO]': 165, '[PAD]': 166}
index_to_token = {idx: token for token, idx in vocab.items()}

cuda


{'[#C]': 0, '[#N]': 1, '[*H0]': 2, '[=*H0]': 3, '[=Branch]': 4, '[=C]': 5, '[=N]': 6, '[=O]': 7, '[=S]': 8, '[Branch]': 9, '[C]': 10, '[Cl]': 11, '[EOS]': 12, '[F]': 13, '[NH0+1]': 14, '[N]': 15, '[OH0-1]': 16, '[O]': 17, '[PAD]': 18, '[PH1]': 19, '[P]': 20, '[Ring1]': 21, '[Ring2]': 22, '[SOS]': 23, '[S]': 24, '[SiH0]': 25, '[pop]': 26}
27
tensor([ 2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([23,  2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([ 2, 10, 17, 10,  2, 12, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])


In [2]:
def select_model(choice, latent):
    if choice == "Trans_MHA":
        from models.Trans_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cpu().eval()
        model.decoder.cpu().eval
        prior = PriorNet(y_dim=3, latent_dim=latent).cpu().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)

    # Trans
    elif choice == "Trans":
        from models.Trans import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cpu().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256_no_mha.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior_no_mha.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    # LSTM
    elif choice == "LSTM":
        from models.LSTM import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)


    # LSTM + MHA
    elif choice == "LSTM_MHA":
        from models.LSTM_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    

    return model, prior

In [3]:
choice = "Trans"
latent_dim = 128
model, prior = select_model(choice, latent_dim)

In [4]:


import torch.nn.functional as F
import numpy as np
from rdkit import Chem, DataStructs

#####################################################################
# Utility: SMILES ↔ Fingerprint & Tanimoto
#####################################################################

def _smiles_to_fp(sm: str):
    mol = Chem.MolFromSmiles(sm)
    return Chem.RDKFingerprint(mol) if mol is not None else None


def tanimoto(sm1: str, sm2: str) -> float:
    fp1, fp2 = _smiles_to_fp(sm1), _smiles_to_fp(sm2)
    if fp1 is None or fp2 is None:
        return 0.0
    return DataStructs.TanimotoSimilarity(fp1, fp2)

#####################################################################
# Beam‑search decoder (temperature + length‑norm)
#####################################################################

def _beam_search_decode(model, z, sos_id: int, eos_id: int, *,
                        beam_width: int = 20, max_len: int = 128,
                        len_penalty: float = 1.0, vocab_size: int = 9999,
                        temperature: float = 0.9, alpha: float = 0.7):
    """Return list[List[int]] without SOS. OOV ids filtered."""
    device = z.device
    batch  = z.size(0)
    out_tokens = []
    for b in tqdm(range(batch)):
        beams = [([sos_id], 0.0, 0.0)]  # (seq, raw_logp, norm_score)
        finished = []
        for _ in range(max_len):
            new_beams = []
            for seq, raw_lp, _ in beams:
                if seq[-1] == eos_id:
                    finished.append((seq, raw_lp))
                    continue
                hidden = model.decoder(
                    torch.tensor(seq, device=device).unsqueeze(0),
                    z[b:b+1]
                )
                logits = model.predict(hidden)[:, -1] / temperature  # sharpen
                logp   = F.log_softmax(logits, dim=-1).squeeze(0)
                topk_val, topk_idx = torch.topk(logp, k=min(beam_width*2, logp.size(0)))
                # filter OOV
                cand = [(tid, tv) for tid, tv in zip(topk_idx.tolist(), topk_val.tolist()) if tid < vocab_size]
                for tid, tv in cand[:beam_width]:
                    new_seq   = seq + [tid]
                    new_rawlp = raw_lp + tv
                    norm_lp   = new_rawlp / (len(new_seq) ** alpha)
                    new_beams.append((new_seq, new_rawlp, norm_lp))
            if not new_beams:
                break
            new_beams.sort(key=lambda x: x[2], reverse=True)
            beams = new_beams[:beam_width]
        if finished:
            for seq, raw_lp in finished:
                beams.append((seq, raw_lp, raw_lp / (len(seq) ** alpha)))
        best_seq = max(beams, key=lambda x: x[1] / (len(x[0]) ** len_penalty))[0]
        # strip SOS/EOS
        best_seq = best_seq[1: best_seq.index(eos_id)] if eos_id in best_seq else best_seq[1:]
        out_tokens.append(best_seq)
    return out_tokens

#####################################################################
# Reconstruction with z = μ  (greedy or beam)
#####################################################################

def reconstruct_zmu(model, dataloader, vocab: dict, *,
                     beam_width: int = 1, len_penalty: float = 1.0,
                     max_len: int | None = None,
                     temperature: float = 0.9,
                     alpha: float = 0.7):
    """Return (mean_tanimoto, num_pairs). beam_width=1 → greedy."""
    model.eval()
    model.cpu()
    tanis, pairs = [], 0
    device = next(model.parameters()).device

    SOS, EOS, PAD = vocab['[SOS]'], vocab['[EOS]'], vocab['[PAD]']
    if max_len is None:
        max_len = 128

    for enc_in, _, _, props in dataloader:
        enc_in, props = enc_in.to(device), props.to(device)
        with torch.no_grad():
            prop_e  = model.input_embedding(props)
            encoded = model.encoder(enc_in, prop_e)
            mu_q    = model.to_means(encoded)
            z       = mu_q  # z = μ

            if beam_width == 1:
                # Greedy ------------------------------------------------------
                dec_in = torch.full((enc_in.size(0), 1), SOS, device=device)
                done   = torch.zeros(enc_in.size(0), dtype=torch.bool, device=device)
                out_tok= [[] for _ in range(enc_in.size(0))]
                for _ in range(max_len):
                    hidden = model.decoder(dec_in, z)
                    logits = model.predict(hidden)[:, -1] / temperature
                    next_tok = logits.argmax(-1, keepdim=True)
                    dec_in = torch.cat([dec_in, next_tok], dim=1)
                    for i, tok in enumerate(next_tok.squeeze(1).tolist()):
                        if not done[i]:
                            if tok == EOS:
                                done[i] = True
                            else:
                                out_tok[i].append(tok)
                    if done.all():
                        break
            else:
                # Beam --------------------------------------------------------
                out_tok = _beam_search_decode(model, z, SOS, EOS,
                                              beam_width=beam_width,
                                              max_len=max_len,
                                              len_penalty=len_penalty,
                                              vocab_size=len(vocab),
                                              temperature=temperature,
                                              alpha=alpha)
        # SELFIES → SMILES & Tanimoto ----------------------------------------
        for r_tok, o_tok in zip(enc_in.cpu().tolist(), out_tok):
            ref_sm = tok_ids_to_smiles([t for t in r_tok if t not in (PAD,)])
            out_sm = tok_ids_to_smiles([t for t in o_tok if t not in (PAD,)])
            if ref_sm and out_sm:
                tanis.append(tanimoto(ref_sm, out_sm))
                pairs += 1
    return np.mean(tanis) if tanis else 0.0, pairs


# 1) 길이 페널티 키워서 긴 후보 선호
print(reconstruct_zmu(model, val_dataloader, dataset.vocab,
                      beam_width=30 , len_penalty=0.9, temperature=0.80,
                      max_len=dataset.max_len+3))

# 150/8 -> 0.4704

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

(np.float64(0.6250621103791515), 1567)
